In [1]:
import torch
import numpy as np
import pandas as pd
from vit_pytorch import ViT
from dataloader import testing_dataset
from util import util
from torch.nn.functional import sigmoid
from torch.utils.data import Dataset, DataLoader

In [54]:
torch.cuda.is_available()

True

In [ ]:
dataset = DataLoader(testing_dataset, shuffle=True, batch_size=128)

In [55]:
vision_model = ViT(
    image_size=128 * 3,
    patch_size=32,
    num_classes=39,
    dim=512,
    depth=6,
    heads=16,
    mlp_dim=1024,
    # dropout = 0.1,
    # emb_dropout = 0.1
).to('cuda')

In [56]:
model_dict = torch.load('./saves/exp_7/model_epoch_100.pth')
model_dict.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'loss', 'testing_loss'])

In [57]:
vision_model.load_state_dict(model_dict['model_state_dict'])
vision_model.eval();

In [58]:
@torch.no_grad()
def calculate_precession(y_hat: torch.Tensor, y: torch.Tensor, pooling = True):
    y = y.cpu()
    y_hat = y_hat.cpu()

    classes_hat = y_hat.apply_(lambda x: 1 if x > .7 else 0)
    y_complement = y.apply_(lambda x: 0 if x == 1 else 0)

    acc = ((classes_hat * y).sum(dim = -1) - (classes_hat * y_complement).sum(dim=-1)) / y.sum(dim=-1)

    return torch.mean(acc) if pooling else acc

In [73]:
sample = next(iter(dataset))

In [74]:
with torch.no_grad():
    probs = vision_model(sample[0])

In [75]:
probs = sigmoid(probs)
probs

tensor([[0.7679, 0.6079, 0.1992,  ..., 0.0927, 0.0915, 0.1209],
        [0.7889, 0.7424, 0.1862,  ..., 0.0768, 0.0814, 0.0922],
        [0.7978, 0.7177, 0.1889,  ..., 0.0809, 0.0837, 0.1028],
        ...,
        [0.8026, 0.7247, 0.1948,  ..., 0.0759, 0.0828, 0.0919],
        [0.7937, 0.7588, 0.1908,  ..., 0.0751, 0.0850, 0.0853],
        [0.8052, 0.7256, 0.1856,  ..., 0.0758, 0.0776, 0.0944]],
       device='cuda:0')

In [76]:
sample[1]

tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]], device='cuda:0')

In [77]:
y = sample[1].cpu().clone()
y_hat = probs.cpu().clone()

classes_hat = y_hat.apply_(lambda x: 1 if x > .7 else 0)
y_complement = y.apply_(lambda x: 0 if x == 1 else 1)

acc = ((classes_hat * y).sum(dim = -1) - (classes_hat * y_complement).sum(dim=-1))

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.]])
tensor([0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0.,
        1., 0., 0., 2., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 2., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
        0., 1., 0., 1., 2., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
        0., 0.])
tensor([[0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 1.,  ..., 1., 1., 1.],
        [0., 0., 1.,  ..., 1., 1., 1.],
        ...,
        [0., 1., 1.,  ..., 1., 1., 1

In [78]:
acc.shape

torch.Size([128])

In [79]:
acc.sum()

tensor(0.)

In [35]:
prec

tensor(nan)

In [ ]:
-0.00034 * torch.pow(prec, 2) + .5 if prec < .5 else .5 * torch.pow(prec, 2) + .5